<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Remote Spark

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor the German Credit Risk model. Use the notebook to enable quality, drift_v2, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. The common configuration JSON, Drift V2 Configuration and Explainability Configuration generated by using the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb).
2. Feedback, payload, explanations queue and result tables in an IBM DB2 storage that use the data description language statements (DDLs) that are generated as part of running the previous common configuration notebook.

In [ ]:
# ----------------------------------------------------------------------------------------------------
# IBM Confidential
# OCO Source Materials
# 5737-H76
# Copyright IBM Corp. 2021-2025
# The source code for this Notebook is not published or other-wise divested of its trade
# secrets, irrespective of what has been deposited with the U.S.Copyright Office.
# ----------------------------------------------------------------------------------------------------

VERSION = "remote_spark-jdbc-1.0"

# Version History
# remote_spark-jdbc-1.0 : Add partitioning information
# <no-verion>  : Initial release

## Contents

1. [Setup](#setup)
2. [Configure Watson OpenScale](#openscale)
3. [Set up a subscription](#subscription)
4. [Quality monitoring](#quality)
5. [Drift V2 monitoring](#drift)
6. [Fairness monitoring](#fairness)
7. [Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

## Package installation

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.



In [1]:
import warnings
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

In [ ]:
%pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

# When this notebook is to be run on RT 24.1 environment, 
# uncomment the below line and execute.
# !pip install --upgrade "matplotlib~=3.10.1" transformers numexpr bottleneck opencv-python transformers numexpr bottleneck opencv-python --no-cache | tail -n 1

In [3]:
%pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.41
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /Users/nelwin/Library/Python/3.10/lib/python/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:



In [4]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [5]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### Spark Cluster

Make sure that the Apache Spark manager on the Spark cluster is running, and then provide the following details:

- SPARK_ENGINE_ENDPOINT: _Endpoint URL where the Spark Manager Application is running_
- SPARK_ENGINE_USERNAME: _Username to connect to Spark Manager Application_
- SPARK_ENGINE_PASSWORD: _Password to connect to Spark Manager Application_
- SPARK_ENGINE_NAME: _Custom display name for the Spark Manager Application_
- SPARK_ENGINE_DESCRIPTION: _Custom description for the Spark Manager Application_

In [6]:
SPARK_ENGINE_NAME=""
SPARK_ENGINE_DESCRIPTION=""
SPARK_ENGINE_ENDPOINT=""
SPARK_ENGINE_ENDPOINT_USERNAME=""
SPARK_ENGINE_ENDPOINT_PASSWORD=""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:


- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_  
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [7]:
spark_parameters = {
    "max_num_executors": 2,
    "min_num_executors": 1,
    "executor_cores": 3,
    "executor_memory": 2,
    "driver_cores": 2,
    "driver_memory": 2
}

### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.

In [8]:
JDBC_CONNECTION_NAME = ""
JDBC_CONNECTION_DESCRIPTION = ""

JDBC_HOST = ""
JDBC_PORT = ""
JDBC_USE_SSL = ""
JDBC_SSL_CERTIFICATE = ""
JDBC_DRIVER = ""
JDBC_USERNAME = ""
JDBC_PASSWORD = ""
JDBC_DATABASE_NAME = ""

In [9]:
num_partitions_recommended = 12

if spark_parameters:
    executors = spark_parameters.get("max_num_executors", 2)
    cores = spark_parameters.get("executor_cores", 2)
    num_partitions_recommended = 2 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

12 is the recommended value for number of partitions in your data. Please change this value as per your data.


- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`.
- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [10]:
PARTITION_COLUMN = "wos_partition_column"
NUM_PARTITIONS = num_partitions_recommended

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [11]:
#feedback

FEEDBACK_SCHEMA_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload table metadata
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_


In [ ]:
#payload logging

PAYLOAD_SCHEMA_NAME = None
PAYLOAD_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_SCHEMA_NAME: _Schema name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [13]:
#explainability
EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_SCHEMA_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [14]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])

### Display Watson OpenScale datamart details

In [15]:
wos_client.data_marts.show()
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id=WOS_CREDENTIALS["instance_id"]

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-11-28 19:42:06.911000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [16]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [17]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider ad4269f3-85e7-49bd-895d-c335f2e6a555 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




None,active,GCR-DriftV2-2,custom_machine_learning,2025-01-20 09:54:24.664000+00:00,ad4269f3-85e7-49bd-895d-c335f2e6a555
None,active,Biswas-Drift-V2-Batch,custom_machine_learning,2025-01-20 09:11:43.297000+00:00,610cffee-a5e9-4bdc-9fcb-832d02cb63bf
None,active,DriftV2Batch,custom_machine_learning,2025-01-17 12:52:49.763000+00:00,19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d
None,active,DRIFT-V2-BATCH-PROVIDER-CPD,custom_machine_learning,2025-01-13 06:14:05.063000+00:00,f8200240-5b59-4c2a-b80b-955ffea9708a
None,active,DriftV2 SDK Support,custom_machine_learning,2025-01-07 15:32:20.255000+00:00,652983c8-67e5-48d5-a852-fc686e08d49b
99999999-9999-9999-9999-999999999999,active,shreya-space,watson_machine_learning,2025-01-07 05:40:32.562000+00:00,aa11c920-1c84-4036-9ddf-e3ca53f62e25
None,active,DRIFT-V2-BATCH-PROVIDER,custom_machine_learning,2024-12-02 07:57:43.792000+00:00,4f82b928-48ec-49e1-a501-25920387e323
None,active,CUSTOM_APIKEY_CLOUD_WITHOUTAPI_PREPROD,custom_machine_learning,2024-12-02 05:35:44.191000+00:00,8f6039aa-cd0a-40b0-a9df-5fe9f4c4d129
None,active,CUSTOM_APIKEY_CLOUD_WITHOUTAPI,custom_machine_learning,2024-12-02 05:35:44.039000+00:00,f9d1dfaf-b3f6-44dd-866b-1e5ea358af4a
4ca7ec1c-6b35-48ef-a45c-0434c6985058,active,MRM_WMLV4_CLOUD_PREPROD,watson_machine_learning,2024-12-02 05:35:43.871000+00:00,a9b82a25-dd5b-473e-ac91-19ef0fcfe367


Note: First 10 records were displayed.


In [ ]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

### Create integrated systems for Spark Engine and JDBC Storage

In [19]:
# Delete existing spark and jdbc integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (SPARK_ENGINE_NAME, JDBC_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system SAMPLE_PG_2


#### Spark Engine

In [ ]:
spark_engine_details = IntegratedSystems(wos_client).add(
    name=SPARK_ENGINE_NAME,
    description=SPARK_ENGINE_DESCRIPTION,
    type="spark",
    credentials={
        "username": SPARK_ENGINE_ENDPOINT_USERNAME,
        "password": SPARK_ENGINE_ENDPOINT_PASSWORD
    },
    connection={
        "endpoint": SPARK_ENGINE_ENDPOINT,
        "location_type": "custom"
    }
).result

spark_engine_id = spark_engine_details.metadata.id
print(spark_engine_details)

#### JDBC Storage

In [ ]:
jdbc_url = "jdbc:db2://{}:{}/{}".format(JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

jdbc_connection_details = IntegratedSystems(wos_client).add(
    name=JDBC_CONNECTION_NAME,
    description=JDBC_CONNECTION_DESCRIPTION,
    type="jdbc",
    credentials={
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
        "jdbc_url": jdbc_url
    },
    connection={
        "location_type": "jdbc",
        "db_driver": JDBC_DRIVER,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
    }
).result

jdbc_connection_id=jdbc_connection_details.metadata.id
print(jdbc_connection_details)

# 3. Set up a subscription <a name="subscription"></a>

In [22]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

73d92434-3988-40fb-88b3-139ff9a3d70b,model,Biswas-GCR-Drift-V2-Batch,00000000-0000-0000-0000-000000000000,aa23ebcd-779c-4cdb-a69f-2b075c2b0df3,Biswas-GCR-Drift-V2-Batch,610cffee-a5e9-4bdc-9fcb-832d02cb63bf,preparing,2025-01-20 09:30:46.782000+00:00,4a805a3a-7799-4476-9a73-f7a46ade7d19
7870a295-3731-4c97-84e4-1a80e378825b,model,GCR-DriftV2Batch,00000000-0000-0000-0000-000000000000,4f2c39c2-ea97-4d69-b327-2637e3d6b39f,GCR-DriftV2Batch,19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d,error,2025-01-17 12:53:01.412000+00:00,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
addfb96c-db7a-4961-932c-eec048be3df4,model,Biswas-GCR-Drift-V2-Batch,00000000-0000-0000-0000-000000000000,f5575a22-7ba9-4b8a-ac07-68fe1be0821c,Biswas-GCR-Drift-V2-Batch,610cffee-a5e9-4bdc-9fcb-832d02cb63bf,error,2025-01-20 09:29:11.780000+00:00,6b16e55d-5edf-4f33-ae5a-2bedad0cc116
bbd7e23e-06ae-4b92-960f-441ce7fb8cd3,model,SDK Batch-subscription,00000000-0000-0000-0000-000000000000,821524c9-d35d-4e2e-bf20-754008f8ef0d,SDK Batch-subscription,f8200240-5b59-4c2a-b80b-955ffea9708a,error,2025-01-20 03:49:24.403000+00:00,debc3aaa-8df6-4564-acc7-9777524750b9
49bbf658-2467-40df-b509-a26ab66818f7,model,DriftV2Batch-SDK-Test,00000000-0000-0000-0000-000000000000,95e02b12-e849-4a0f-8922-544dc1178837,DriftV2Batch-SDK-Test,652983c8-67e5-48d5-a852-fc686e08d49b,active,2025-01-07 15:33:24.779000+00:00,ad7b6384-d2a1-4dde-b66e-b081580cb5c4
ac369058-f1cc-484d-bf13-f1a6d8062b53,model,[asset] Drift-Batch,00000000-0000-0000-0000-000000000000,02247a02-1d58-4b05-930b-f04ea167f33a,Drift-Batch,f8200240-5b59-4c2a-b80b-955ffea9708a,active,2025-01-16 09:35:45.225000+00:00,e436e53d-d385-4c15-84f3-1a0283920a62
5a84ca44-395b-4f1f-9238-28728148d146,model,[asset] Drift-Batch,00000000-0000-0000-0000-000000000000,1d7c4edd-2a05-4d49-9e59-c651e2a10aff,Drift-Batch,f8200240-5b59-4c2a-b80b-955ffea9708a,active,2025-01-16 05:46:47.478000+00:00,3e492d41-bba1-47d4-ab60-1a69090c36df
a0a602a0-9dae-42fd-92a3-f0be695c5f51,model,DriftV2Batch-SDK-Test,00000000-0000-0000-0000-000000000000,4c56bfa9-35a8-4169-9fed-db9f53b20cc2,DriftV2Batch-SDK-Test,652983c8-67e5-48d5-a852-fc686e08d49b,active,2025-01-07 15:34:07.611000+00:00,1cbc4b58-3374-4472-8556-841371719afa
f972ffb9-13c7-4e7b-8081-c400244231fa,model,GCR - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,d6777bec-f644-4068-a2bb-293675c312c8,gcr-shreya,aa11c920-1c84-4036-9ddf-e3ca53f62e25,active,2025-01-07 05:40:43.762000+00:00,4ea84707-a0e0-4ad3-a7a6-21c47fa82cec
fcd58b54-c281-449c-938f-8c20632d7671,model,GermanCreditRiskModelICP,00000000-0000-0000-0000-000000000000,d31e0810-b1ad-457f-bc91-a5aedf40d26c,GermanCreditRiskModelICP,85fcd6fc-754a-4ded-8cc2-969821bd4489,active,2024-12-30 06:36:07.118000+00:00,541e7e69-06e9-4a99-bde2-db84c5fb11bd


Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, type a path to the common configuration JSON file that you created by running the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb). After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [ ]:
import uuid
import json

common_configuration = None
with open("/path/to/dir/containing/configuration.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        print("Please provide the correct path to the common configuration JSON")
    
# Set asset details
asset = Asset(
    asset_id=str(uuid.uuid4()),
    url="",
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=str(uuid.uuid4()),
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type="batch"
)

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=[common_configuration["probability"]],
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"],
    training_data_schema=SparkStruct.from_dict(common_configuration["training_data_schema"]),
    input_data_schema=SparkStruct.from_dict(common_configuration["input_data_schema"]),
    output_data_schema=SparkStruct.from_dict(common_configuration["output_data_schema"])
)

# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_engine_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_SCHEMA_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=JDBC_DATABASE_NAME, 
        schema_name=FEEDBACK_SCHEMA_NAME, 
        table_name=FEEDBACK_TABLE_NAME, 
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_SCHEMA_NAME is not None and PAYLOAD_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=JDBC_DATABASE_NAME, 
        schema_name=PAYLOAD_SCHEMA_NAME, 
        table_name=PAYLOAD_TABLE_NAME, 
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    
if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue",
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result",
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [24]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    data_sources=data_sources).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
    "url": "/v2/subscriptions/52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
    "created_at": "2025-01-20T09:54:38.907000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "ad4269f3-85e7-49bd-895d-c335f2e6a555",
    "asset": {
      "asset_id": "0e64aa20-836a-42e9-8401-273ff05d973e",
      "url": "",
      "name": "GCR-DriftV2-2",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "label_column": "Risk",
      "prediction_field": "prediction",
      "feature_fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavin

In [26]:
# Check subscription status
import time
from datetime import datetime
subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

15:25:54 preparing
15:26:10 preparing
15:26:26 preparing
15:26:42 active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [27]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 100
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "url": "/v2/monitor_instances/5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "created_at": "2025-01-20T09:57:12.707000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [28]:
quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
print(datetime.now().strftime("%H:%M:%S"), quality_status)

15:27:15 active


In [29]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "url": "/v2/monitor_instances/5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "created_at": "2025-01-20T09:57:12.707000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-20T09:57:13.156000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

### Run an on-demand evaluation

In [30]:
# Check Quality monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "url": "/v2/monitor_instances/5fc73737-02b7-4567-a0fe-4c8b2d8079ae",
    "created_at": "2025-01-20T09:57:12.707000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-20T09:57:13.156000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

In [31]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "33487920-221d-4bce-842d-a99ddcab6709",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:33487920-221d-4bce-842d-a99ddcab6709",
    "url": "/v2/monitor_instances/5fc73737-02b7-4567-a0fe-4c8b2d8079ae/runs/33487920-221d-4bce-842d-a99ddcab6709",
    "created_at": "2025-01-20T09:57:20.573000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2025-01-20T09:57:20.568000Z",
      "started_at": "2025-01-20T09:57:20.573000Z",
      "operators": []
    }
  }
}


In [32]:
# Check run status

quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_run_status)

15:27:21 running
15:27:51 running
15:28:34 running
15:29:05 running
15:29:36 running
15:30:06 running
15:30:37 running
15:31:08 running
15:31:38 running
15:32:09 running
15:32:40 finished


### Display quality metrics

In [33]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2025-01-20 10:02:22.385271+00:00,true_positive_rate,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.9,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,area_under_roc,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.7227272727272727,0.8,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,precision,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.8780487804878049,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,f1_measure,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.888888888888889,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,accuracy,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.8235294117647058,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,log_loss,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.4702648897572855,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,false_positive_rate,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.45454545454545453,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,area_under_pr,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.8733620277379245,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 10:02:22.385271+00:00,recall,93c95e96-eaf4-4d1b-99fa-d83095ec7953,0.9,None,None,['model_type:original'],quality,5fc73737-02b7-4567-a0fe-4c8b2d8079ae,33487920-221d-4bce-842d-a99ddcab6709,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86


# 6. Drift V2 monitoring <a name="drift_v2"></a>

### Enable the drift v2 monitor

In the following code cell, type a path to the drift v2 configuration tar ball.

In [41]:
# Upload drift_v2 archive

wos_client.monitor_instances.upload_drift_v2_archive(
    archive_path="/path/to/dir/containing/drift_v2_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{}

In [42]:
# Delete existing drift_v2 monitor

monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift v2 monitor instance with id: ', monitor_instance.metadata.id)

In [43]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 100,
        "max_samples": 1000,
        "train_archive": False,
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'efa13475-9c04-4d2b-ac1d-838ea289e780'

### Check monitor instance status

In [44]:
drift_v2_status = None

while drift_v2_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
    drift_v2_status = monitor_instance_details.entity.status.state
    if drift_v2_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)

16:18:57 preparing
16:19:28 active


### Trigger on-demand monitor run

In [45]:
# Check Drift v2 monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "efa13475-9c04-4d2b-ac1d-838ea289e780",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:efa13475-9c04-4d2b-ac1d-838ea289e780",
    "url": "/v2/monitor_instances/efa13475-9c04-4d2b-ac1d-838ea289e780",
    "created_at": "2025-01-20T10:48:11.739000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-20T10:49:01.333000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
   

In [46]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "71d3d6e5-5f7a-45a4-95b7-49180e738792",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:71d3d6e5-5f7a-45a4-95b7-49180e738792",
    "url": "/v2/monitor_instances/efa13475-9c04-4d2b-ac1d-838ea289e780/runs/71d3d6e5-5f7a-45a4-95b7-49180e738792",
    "created_at": "2025-01-20T10:49:29.181000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
        },
        "data_sets": {
          "drift_insights": "de78458b-9893-43bc-aff1-5cb0f2df61da",
          "drift_intervals": "cf79e9c2-e84e-4d8a-9b6d-9bb02cd7915c",
          "drift_scores": "9bbdc8ca-e9a0-44de-bdd7-44acd2b17107",
          "drift_

In [47]:
# Check run status
drift_v2_run_status = None
while drift_v2_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_v2_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_v2_run_status = monitoring_run_details.entity.status.state
    if drift_v2_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)



17:04:49 finished


### Check Drift V2 metrics

In [48]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2025-01-20 11:14:09.058664+00:00,records_processed,40f7d3d2-24a5-49fb-996b-4bc118bed0ba,13.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:feature', 'field_name:Dependents']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:09.058664+00:00,feature_drift_score,40f7d3d2-24a5-49fb-996b-4bc118bed0ba,0.0573,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:feature', 'field_name:Dependents']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:12.375630+00:00,records_processed,b9ea4cbc-2348-4169-bfd4-f1ec96f52a3f,13.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:feature', 'field_name:Telephone']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:12.375630+00:00,feature_drift_score,b9ea4cbc-2348-4169-bfd4-f1ec96f52a3f,0.0055,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:feature', 'field_name:Telephone']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:22.341728+00:00,records_processed,8afdd63e-25c5-495d-a3fd-6a9f5900b65a,13.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:feature', 'field_name:ForeignWorker']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:22.341728+00:00,feature_drift_score,8afdd63e-25c5-495d-a3fd-6a9f5900b65a,0.141,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:feature', 'field_name:ForeignWorker']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:28.674112+00:00,records_processed,158adeb9-dac5-49f0-9d75-b5bc0152b6a7,13.0,None,None,"['computed_on:payload', 'field_type:subscription']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:28.674112+00:00,confidence_drift_score,158adeb9-dac5-49f0-9d75-b5bc0152b6a7,0.9217,None,0.05,"['computed_on:payload', 'field_type:subscription']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:28.674112+00:00,feature_drift_score,158adeb9-dac5-49f0-9d75-b5bc0152b6a7,0.2182,None,0.05,"['computed_on:payload', 'field_type:subscription']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:14:28.674112+00:00,prediction_drift_score,158adeb9-dac5-49f0-9d75-b5bc0152b6a7,0.1496,None,0.05,"['computed_on:payload', 'field_type:subscription']",drift_v2,efa13475-9c04-4d2b-ac1d-838ea289e780,71d3d6e5-5f7a-45a4-95b7-49180e738792,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86


Note: First 10 records were displayed.


# 7. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [49]:
# Delete existing fairness monitor, if any.
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "fairness" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing fairness monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing fariness monitor instance with id:  db3ad74c-a974-440a-8bb2-65f41e5844f0


In [50]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = configuration_json["fairness_configuration"]["parameters"]
thresholds = configuration_json["fairness_configuration"]["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "87d4df4d-423f-43af-9ce3-fa2532121afc",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:87d4df4d-423f-43af-9ce3-fa2532121afc",
    "url": "/v2/monitor_instances/87d4df4d-423f-43af-9ce3-fa2532121afc",
    "created_at": "2025-01-20T11:40:01.103000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "training_data_last_processed_time": "2025-01-17T09:40:55.219953Z",
      "min_records": 100,
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "av

### Check monitor instance status

In [51]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

11:40:04 preparing
11:40:35 active


### Run an on-demand evaluation

In [52]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "26e5f97c-ebf3-4e3d-a458-245516482e5c",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:26e5f97c-ebf3-4e3d-a458-245516482e5c",
    "url": "/v2/monitor_instances/87d4df4d-423f-43af-9ce3-fa2532121afc/runs/26e5f97c-ebf3-4e3d-a458-245516482e5c",
    "created_at": "2025-01-20T11:41:41.486000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "average_abs_odds_difference",
            "false_negative_rate_difference",
            "false_positive_rate_difference",
            "false_discovery_rate_difference",
            "false_omission_rate_differenc

In [53]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

11:41:45 running
11:42:15 running
11:42:46 running
11:43:16 running
11:43:47 running
11:44:17 running
11:44:48 running
11:45:18 running
11:45:49 running
11:46:19 running
11:46:50 running
11:47:20 finished


### Display fairness metrics

In [54]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2025-01-20 11:46:44.348010+00:00,false_positive_rate_difference,3e851a00-1331-492e-9028-355fa61970d0,0.857,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,false_negative_rate_difference,3e851a00-1331-492e-9028-355fa61970d0,-0.174,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,error_rate_difference,3e851a00-1331-492e-9028-355fa61970d0,0.023,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,false_omission_rate_difference,3e851a00-1331-492e-9028-355fa61970d0,-0.4,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,fairness_value,3e851a00-1331-492e-9028-355fa61970d0,150.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,statistical_parity_difference,3e851a00-1331-492e-9028-355fa61970d0,0.333,-0.3,0.3,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,false_discovery_rate_difference,3e851a00-1331-492e-9028-355fa61970d0,0.14,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,average_odds_difference,3e851a00-1331-492e-9028-355fa61970d0,0.516,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,average_abs_odds_difference,3e851a00-1331-492e-9028-355fa61970d0,0.516,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-20 11:46:44.348010+00:00,false_positive_rate_difference,3e851a00-1331-492e-9028-355fa61970d0,-0.455,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,87d4df4d-423f-43af-9ce3-fa2532121afc,26e5f97c-ebf3-4e3d-a458-245516482e5c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86


Note: First 10 records were displayed.


# 8. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [55]:
with open("/path/to/dir/containing/explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability configuration archive successfully.")

Uploaded explainability configuration archive successfully.


In [56]:
# Delete existing explainability monitor.
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "explainability" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing explainability monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing explainability monitor instance with id:  d85fa772-a22c-4830-a7bf-3bbf3fc75540


In [57]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    # Comment the below lines to disable lime global explanation
    # Lime global explanation is available from Cloud Pak for Data version 4.6.4 onwards.
    "global_explanation": {
        "enabled": True,  # Flag to enable global explanation 
        "explanation_method": "lime",
        # "sample_size": 10, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "bfa372bb-c0dc-4aee-b586-6796a158e4b3",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:bfa372bb-c0dc-4aee-b586-6796a158e4b3",
    "url": "/v2/monitor_instances/bfa372bb-c0dc-4aee-b586-6796a158e4b3",
    "created_at": "2025-01-21T07:39:55.431000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "global_explanation": {
        "enabled": true,
        "explanation_method": "lime",
        "sample_size": 10
      }
    },
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "s

### Check monitor instance status

In [58]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

07:02:47 active


### Run an on-demand evaluation

In [59]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "d85fa772-a22c-4830-a7bf-3bbf3fc75540",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:d85fa772-a22c-4830-a7bf-3bbf3fc75540",
    "url": "/v2/monitor_instances/d85fa772-a22c-4830-a7bf-3bbf3fc75540",
    "created_at": "2025-01-20T14:20:33.112000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-20T14:24:24.206000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86"
    },
    "parameters": {
      "config_modified_at": "2025-01-20T14:24:24.142583Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "global_explanation": {
        "enabled": 

In [60]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "bc4264e4-204e-49e1-a2b8-f6641e2c4da8",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:bc4264e4-204e-49e1-a2b8-f6641e2c4da8",
    "url": "/v2/monitor_instances/bfa372bb-c0dc-4aee-b586-6796a158e4b3/runs/bc4264e4-204e-49e1-a2b8-f6641e2c4da8",
    "created_at": "2025-01-21T09:23:22.603000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_modified_at": "2025-01-21T09:21:44.098205Z",
      "config_package_file": "explainability.tar (1).gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "global_explanation": {
        "aggregation_method": "mean_abs",
        "aggregation_methods": [
          "mean_abs"
        ],
        "baseline_explanation_measurement_id": "f4714c1f-af0f-4469-bdd6-053bf0a875e8",
        "distribution_features_count": 100,
        "distribution_size": 1000,

In [61]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

09:23:26 running
09:24:26 running
09:25:26 running
09:26:27 running
09:27:27 running
09:28:28 running
09:29:28 finished


In [62]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

2025-01-21 09:28:51.894207+00:00,global_explanation_stability,36c987bd-9b8c-41c4-9df7-3bb2350c22a6,0.9795605604404594,0.8,None,['data_set_type:payload'],explainability,bfa372bb-c0dc-4aee-b586-6796a158e4b3,bc4264e4-204e-49e1-a2b8-f6641e2c4da8,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-21 09:21:45.761430+00:00,global_explanation_stability,f4714c1f-af0f-4469-bdd6-053bf0a875e8,1.0,0.8,None,['data_set_type:training'],explainability,bfa372bb-c0dc-4aee-b586-6796a158e4b3,9ac9471e-41e8-4348-aa78-067a5753ea3c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86


### Display sample explanations

In [63]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result
print(explanations)

{
  "total_count": 100,
  "limit": 50,
  "offset": 0,
  "explanation_fields": [
    "explanation_task_id",
    "scoring_id",
    "created_at",
    "finished_at",
    "status",
    "prediction",
    "subscription_id",
    "deployment_id",
    "asset_name",
    "deployment_name",
    "probability",
    "explanation_type"
  ],
  "explanation_values": [
    [
      "3378de3d-ee19-410f-9c97-263fb1a2946f",
      "9820ff2a017cadb6ada7931f23adf6f7-1",
      "2025-01-21T09:24:34.974443Z",
      "2025-01-21T09:24:34.974614Z",
      "finished",
      "No Risk",
      "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
      "fbb6b55b-d40c-4b8b-802b-8a6448450b73",
      "GCR-DriftV2-2",
      "GCR-DriftV2-2",
      0.677657,
      "lime"
    ],
    [
      "fda2b010-4ef4-43e4-b73f-ebe386e58cff",
      "96927a161a0d60c579981e93f8d78fc2-7",
      "2025-01-21T09:24:34.993858Z",
      "2025-01-21T09:24:34.994006Z",
      "finished",
      "No Risk",
      "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
      "fbb6b55b-d4

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale using Remote Apache Spark. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.